In [ ]:
!pip install pysqlite3

In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# from pysqlite3 import dbapi2 as sqlite3
import sqlite3
print(sqlite3.sqlite_version_info)

(3, 45, 3)


In [2]:
import pandas as pd
from IPython.display import display, HTML

In [31]:
dbname = "music_streaming_1.db"

def printSqlResults(cursor, tbl):
  df=pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
  display(HTML("<b><font color=Green>"+tbl+"</font></b>"+df.to_html(index=False)))

def runSql(caption, query):
    conn = sqlite3.connect(dbname)
    cursor = conn.cursor()
    cursor.execute(query)
    
    # Check if the query is a SELECT (fetch results only for SELECT queries)
    if query.strip().upper().startswith("SELECT"):
        printSqlResults(cursor, caption)
    else:
        print(f"Query executed: {caption}")
    
    conn.commit()
    conn.close()

In [32]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")

conn.commit()
conn.close()

In [57]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor=conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

#Insert data into the table

cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")
# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")

OperationalError: database is locked

In [6]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
runSql('Classic Query', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [7]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
runSql('Classic Query with songs like Ye..', qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [8]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
runSql('All genres', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [9]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

runSql('Distinct Genres', qry_distinct)

genre
Pop
Rock
Classic
None


In [10]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [11]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_artist_count)


artist,genre,num_songs
Beatles,Classic,3
DJ,None,1
Ed Sheeran,Rock,3
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [12]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
runSql('One Large Table',qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


In [13]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
runSql('Highly Rated Songs',qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
runSql('Highly Rated Songs',qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


In [14]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id)
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist
ORDER BY COUNT(Listens.song_id) DESC;
"""

runSql('Popular Songs', qry_pop_songs)

song_id,title,artist,count(Listens.song_id)
2,Willow,Taylor Swift,3
1,Evermore,Taylor Swift,2
6,Yesterday,Beatles,2
8,Hey Jude,Beatles,1
7,Yellow Submarine,Beatles,1


In [15]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
runSql('Songs by Ed Sheeran and Taylor Swift', qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
runSql('Songs on both Pop and Rock', qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Bad Blood,Taylor Swift
Evermore,Taylor Swift
Photograph,Ed Sheeran
Shape of You,Ed Sheeran
Shivers,Ed Sheeran
Willow,Taylor Swift


In [16]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
runSql('Songs listened by user_id=1',qry_listen_usr)

title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Yesterday,Beatles


In [17]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""

In [18]:
#ASSIGNMENT-2. To be submitted as a PDF with queries and screenshots.

#1 Insert the above into the recommendations table
#2 Generate the recommendaions for Minnie

#3 Re-do the generation of recommendations now on the basis of listen time
#4 Generate new recommendations
#5 What are the differences with the static method on #2 above

In [19]:
def print_sql_results(cursor, tbl):
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    display(HTML(f"<b><font color=Green>{tbl}</font></b>{df.to_html(index=False)}"))

def run_sql(caption, query):
    conn = sqlite3.connect(dbname)
    cursor = conn.cursor()
    cursor.execute(query)
    print_sql_results(cursor, caption)
    conn.close()

In [20]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()
cursor.execute(qry_sharing)
recommendations = cursor.fetchall()

for rec in recommendations:
    cursor.execute("""
        INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
        VALUES ((SELECT IFNULL(MAX(recommendation_id), 0) + 1 FROM Recommendations), CURRENT_TIMESTAMP, ?, ?);
    """, rec)

conn.commit()

In [21]:
run_sql("Recommendations for Minnie", """
SELECT R.recommendation_id, R.recommendation_time, U.name AS user, S.title AS song
FROM Recommendations R
JOIN Users U ON R.user_id = U.user_id
JOIN Songs S ON R.song_id = S.song_id
WHERE U.user_id = 2;
""")

recommendation_id,recommendation_time,user,song
1,2024-11-23 18:37:38,Minnie,Evermore
2,2024-11-23 18:37:38,Minnie,Yesterday


In [26]:
qry_recommendations_minnie = """
SELECT Songs.title AS "Title", Songs.artist AS "Artist Name", R.recommendation_time
FROM Recommendations R
JOIN Songs ON R.song_id = Songs.song_id
WHERE R.user_id = 2;
"""
runSql('Recommendations for Minnie', qry_recommendations_minnie)

Title,Artist Name,recommendation_time
Evermore,Taylor Swift,2024-11-23 18:37:38
Yesterday,Beatles,2024-11-23 18:37:38


In [58]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

# Insert more users
cursor.execute("""
INSERT INTO Users (user_id, name, email)
VALUES
    (5, 'Donald', 'donald@example.com'),
    (6, 'Goofy', 'goofy@example.com'),
    (7, 'Daisy', 'daisy@example.com'),
    (8, 'Max', 'max@example.com');
""")

# Insert more listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (10, 4, 1, 4.8, '2024-08-25 11:00:00'),
    (11, 4, 3, 4.0, '2024-08-24 14:30:00'),
    (12, 5, 3, 3.7, '2024-08-24 18:20:00'),
    (13, 5, 4, 4.9, '2024-08-23 20:00:00'),
    (14, 6, 5, 4.5, '2024-08-23 09:15:00'),
    (15, 6, 6, 4.0, NULL),
    (16, 7, 8, 4.3, '2024-08-22 08:00:00'),
    (17, 7, 9, 3.8, '2024-08-21 12:45:00'),
    (18, 8, 10, 4.2, '2024-08-20 15:30:00'),
    (19, 8, 7, 3.9, '2024-08-19 10:30:00'),
    (20, 1, 3, 4.3, '2024-08-28 10:00:00'),
    (21, 2, 3, 4.6, '2024-08-28 11:00:00'),
    (22, 3, 4, 4.2, '2024-08-27 09:00:00'),
    (23, 4, 4, 4.1, '2024-08-26 16:30:00'),
    (24, 5, 5, 4.5, '2024-08-25 12:15:00'),
    (25, 6, 5, 4.0, '2024-08-24 14:30:00'),
    (26, 7, 10, 4.3, '2024-08-23 13:00:00'),
    (27, 8, 10, 4.7, '2024-08-22 17:00:00');
""")

conn.commit()
conn.close()

# Verify the updated data
runSql('Users', "SELECT * FROM Users;")
runSql('Listens', "SELECT * FROM Listens;")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com
5,Donald,donald@example.com
6,Goofy,goofy@example.com
7,Daisy,daisy@example.com
8,Max,max@example.com


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None
10,4,1,4.8,2024-08-25 11:00:00


In [59]:
qry_recommendations_time = """
WITH song_similarity_time AS (
  SELECT u1.song_id AS song1, u2.song_id AS song2, COUNT(*) AS common_users
  FROM Listens u1
  JOIN Listens u2
  ON u1.user_id = u2.user_id
  AND u1.song_id != u2.song_id
  WHERE u1.listen_time IS NOT NULL AND u2.listen_time IS NOT NULL
  GROUP BY u1.song_id, u2.song_id
  HAVING COUNT(*) > 1
),
recs_time AS (
  SELECT user_id, song2 AS song_id, datetime('now') AS recommendation_time
  FROM song_similarity_time
  JOIN Listens AS L
  ON L.song_id = song_similarity_time.song1
  WHERE song_similarity_time.song2 NOT IN (
    SELECT song_id FROM Listens AS temp WHERE temp.user_id = L.user_id
  )
)
INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
SELECT ROW_NUMBER() OVER (ORDER BY user_id), recommendation_time, user_id, song_id
FROM recs_time;
"""

runSql('Insert Recommendations (Time-Based)', qry_recommendations_time)
runSql('Updated Recommendations Table', "SELECT * FROM Recommendations;")

Query executed: Insert Recommendations (Time-Based)


recommendation_id,recommendation_time,user_id,song_id
1,2024-11-23 19:22:53,1,4
2,2024-11-23 19:22:53,2,10
3,2024-11-23 19:22:53,2,1
4,2024-11-23 19:22:53,2,4
5,2024-11-23 19:22:53,3,3
6,2024-11-23 19:22:53,3,3
7,2024-11-23 19:22:53,5,1
8,2024-11-23 19:22:53,7,7


In [61]:
qry_updated_recommendations = """
SELECT Users.name, Songs.title, Songs.artist, Recommendations.recommendation_time
FROM Recommendations
JOIN Users ON Recommendations.user_id = Users.user_id
JOIN Songs ON Recommendations.song_id = Songs.song_id;
"""
runSql('New Recommendations', qry_updated_recommendations)

name,title,artist,recommendation_time
Mickey,Photograph,Ed Sheeran,2024-11-23 19:22:53
Minnie,DJ Mix,DJ,2024-11-23 19:22:53
Minnie,Evermore,Taylor Swift,2024-11-23 19:22:53
Minnie,Photograph,Ed Sheeran,2024-11-23 19:22:53
Daffy,Shape of You,Ed Sheeran,2024-11-23 19:22:53
Daffy,Shape of You,Ed Sheeran,2024-11-23 19:22:53
Donald,Evermore,Taylor Swift,2024-11-23 19:22:53
Daisy,Yellow Submarine,Beatles,2024-11-23 19:22:53


In [ ]:
The query you've written for computing recommendations with `qry_sharing` and the method you're comparing it to from **#2 above** (which is likely based on a **static similarity** approach, e.g., based on song features or fixed metrics like popularity) has some key differences.

### **Let's break down the differences:**

### **1. Type of Similarity Calculation:**
- **Your query (`qry_sharing`):**
  - The query looks for **song pairs shared by more than one user**. This approach calculates **co-listening** between users by checking whether two songs have been listened to by the same users, **without any reference to song features or popularity**.
  - It generates recommendations based on **behavioral similarity**, i.e., which songs were listened to by common users. This is a **dynamic method** that uses actual user data, and it may change over time as new data comes in.

- **Static method (as in #2 above):**
  - Typically, in a static recommendation system, the similarity is computed using **predefined factors** such as song attributes (e.g., genre, artist, or even popularity) or fixed **user ratings**.
  - This approach usually doesn't change with time, so the recommendation will be the same every time unless manually updated. For example, if song attributes like genre are used, the system might recommend songs from the same genre regardless of who is listening to them.

### **2. Data Used for Recommendations:**
- **Your query (`qry_sharing`):**
  - Uses the **`Listens`** table to compute recommendations, relying on actual listening behavior (which songs were played by which users). It uses **co-listening** information (how many users listen to both songs) to create similarity between songs.
  - The recommendation is based on **shared behavior**: If two users listen to similar songs, the system assumes they have similar tastes, and suggests songs they've both listened to.

- **Static method (as in #2 above):**
  - Relies on **predefined features** such as **song genre, artist, or ratings** to generate recommendations. This might be based on collaborative filtering, content-based filtering, or even hybrid models that combine these approaches, but they typically don't involve real-time, behavior-based similarity.
  - In contrast to `qry_sharing`, static methods use **song-level data** or **user-level ratings** to generate recommendations without considering real-time user listening activity.

### **3. Output of Recommendations:**
- **Your query (`qry_sharing`):**
  - The output of your query is a list of recommended songs based on **co-listening** by different users. It calculates which songs to recommend based on the behavior of users who listened to the same songs at some point. For example, if two users listen to songs 1 and 2, it might recommend song 2 to the first user (if they haven't already listened to it).
  - The recommendation is **dynamic** and changes depending on the users and songs that are currently being listened to. New data on listening behavior can affect future recommendations.

- **Static method (as in #2 above):**
  - The static method would generate recommendations based on **static, unchanging parameters** like genre or rating (e.g., recommending songs in the same genre, or based on the most highly-rated songs by a specific artist).
  - The recommendations are **static**, meaning that unless the system is updated (e.g., by adding new songs or changing the static parameters), the same recommendations will be repeated.

### **4. Time Sensitivity:**
- **Your query (`qry_sharing`):**
  - Your method is more **time-sensitive** because it dynamically analyzes which songs are listened to by users at the same time. If a user listens to a song today, their recommendations are based on their co-listeners, even if it's a recent event. The recommendations are based on **shared, real-time listening behavior**.

- **Static method (as in #2 above):**
  - The static method typically doesn't incorporate time sensitivity into its model, since it's based on **static features** (such as genre or ratings) or precomputed similarity measures. These recommendations would remain the same regardless of the time when the user listens to songs.

### **5. Data Dependency:**
- **Your query (`qry_sharing`):**
  - The system will only generate recommendations if there is enough **co-listening data**. The more users listen to the same songs, the better the recommendation engine can work. However, if too few users listen to the same songs, or if many songs have sparse listening data, there might be **no recommendations** or lower-quality suggestions.

- **Static method (as in #2 above):**
  - The static method can still generate recommendations based on predefined features, even if there is limited user activity. For example, if you know a song's genre and its ratings, the recommendation engine might still recommend songs from the same genre or similarly rated songs, even if no one has listened to those songs recently.

### **6. Complexity:**
- **Your query (`qry_sharing`):**
  - This query is more **complex** in terms of data processing because it involves joining tables to analyze user behavior and song listening patterns. It also relies on **real-time data** to generate recommendations, which can be computationally intensive.

- **Static method (as in #2 above):**
  - Static methods are generally **less complex** because they don't require real-time analysis or large joins. They usually work with a **fixed dataset** (e.g., genres or ratings), making them faster to compute but less adaptive.

---

### **Summary of Key Differences:**

| Aspect                        | `qry_sharing` (Behavioral)                                      | Static Method (Feature-Based)                            |
|-------------------------------|------------------------------------------------------------------|----------------------------------------------------------|
| **Similarity Calculation**     | Based on **co-listening** behavior between users.               | Based on **song attributes** (genre, artist, rating).     |
| **Dynamic vs. Static**         | **Dynamic** (based on recent user behavior).                   | **Static** (doesn't change unless features are updated).  |
| **Data Used**                  | **User behavior** (which songs were listened to by whom).      | **Song-level data** (genre, artist, ratings).            |
| **Time Sensitivity**           | **Time-sensitive** (based on when songs were listened to).     | **Not time-sensitive** (features remain constant).        |
| **Output Type**                | Recommendations based on **co-listening behavior**.             | Recommendations based on **static features**.             |
| **Dependency on Data**         | Relies on **co-listening data** and real-time user activity.   | Can generate recommendations with **less user activity**.|
| **Complexity**                 | **More complex** (requires joins and real-time data analysis). | **Less complex** (works with static data).                |

### **Conclusion:**
- The query (`qry_sharing`) is an **advanced**, **dynamic** approach to generating recommendations based on user co-listening, and it adjusts over time as more user behavior data becomes available.
- The static method (as described in #2) is **less dynamic** and relies more on **predefined features** like genre or rating, without taking into account the time of listening or current user behavior.

If you need a **more adaptive** system that generates recommendations based on real-time user behavior, then the **co-listening-based method** in `qry_sharing` would be more effective. If you're looking for a **simpler, more static system**, then the feature-based method from #2 would suffice.